<a href="https://colab.research.google.com/github/xvmon234-ai/Learning-Python/blob/main/Financial_Risk_Prediction/04_Machine_Learning_Model_Development/Machine_Learning_Model_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **머신러닝 모델 개발(Machine Learning Model Development)**


---

### **1단계: 데이터 준비: 재무 데이터 재수집 및 비율 계산**
먼저, 이전 단계에서 작성한 코드를 다시 실행하여 모델 학습에 필요한 데이터를 준비합니다.

In [10]:
# 필요한 라이브러리 불러오기
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1 단계: 데이터 수집
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
financial_data = {}

for ticker in tickers:
  try:
    stock = yf.Ticker(ticker)
    income_stmt = stock.income_stmt.T
    balance_sheet = stock.balance_sheet.T
    financial_data[ticker] = {'income_stmt': income_stmt, 'balance_sheet': balance_sheet} # 질문 01. 해당 내용에 대한 추가설명
  except Exception as e:
        print(f"{ticker}의 데이터를 불러오는 중 오류가 발생했습니다: {e}")

# 2 단계: 재무 비율 계산

calculated_ratios = {}

for ticker in tickers:
  if ticker in financial_data:
    income_df = financial_data[ticker]['income_stmt']
    balance_df = financial_data[ticker]['balance_sheet']

    # DataFrame 복사본을 생성하여 오류 방지
    income_df = income_df.copy()
    balance_df = balance_df.copy()

    # 각 기업별 재무 비율 계산
    ratios_df = pd.DataFrame(index=income_df.index)

    # 1. 유동성 비율 (Liquidity Ratios)
    # 유동비율 = 유동자산 / 유동부채: 단기 채무 지급 능력을 나타냄.
    ratios_df['CurrentRatio'] = balance_df['Current Assets'] / balance_df['Current Liabilities']

    # 당좌비율 = (유동자산 - 재고자산) / 유동부채: 재고를 제외한 엄격한 단기 지급 능력.
    # META의 경우 재고자산이 없으므로 재고자산을 0으로 간주하여 계산해야 함
    # 따라서 다음의 조건문을 활용
    if 'Inventory' in balance_df.columns:
      ratios_df['QuickRatio'] = (balance_df['Current Assets'] - balance_df['Inventory']) / balance_df['Current Liabilities']
    # 재고자산이 없는 경우
    # 업종에 따라서 이와 같은 경우가 존재할 수 있음을 유의해야함
    else:
      ratios_df['QuickRatio'] = balance_df['Current Assets'] / balance_df['Current Liabilities']

    # 2. 수익성 비율 (Profitability Ratios)
    # 매출총이익률 = 매출총이익 / 매출(OperatingRevenue): 주된 영업활동을 통해 얻은 수익성.
    ratios_df['GrossProfitMargin'] = income_df['Gross Profit'] / income_df['Operating Revenue']

    # 영업이익률 = 영업이익 / 매출(Operating Revenue): 핵심 사업의 효율성.
    ratios_df['OperatingProfitMargin'] = income_df['Operating Income'] / income_df['Operating Revenue']

    # 3. 레버리지 비율 (Leverage Ratios)
    # 부채비율 = 총부채 / 총자본(Common Stock Equity): 타인자본 의존도를 나타냄.
    ratios_df['DebtRatio'] = balance_df['Total Debt'] / balance_df['Common Stock Equity']

# 3단계: 목표 변수(Target) 생성
    # 다음 해에 순이익이 음수이면 '1' (재무 위험), 아니면 '0' (정상)으로 정의
    # shift(-1)을 사용하여 다음 연도 데이터를 가져옴
    net_income_next_year = income_df['Net Income'].shift(-1)
    ratios_df['FinancialRisk'] = (net_income_next_year < 0).astype(int)

    # 계산된 재무 비율 저장
    calculated_ratios[ticker] = ratios_df

all_ratios_df = pd.concat(calculated_ratios, names=['Ticker', 'Date'])
all_ratios_df = all_ratios_df.reset_index()

# 결측치 제거
all_ratios_df = all_ratios_df.dropna()

print("✅ 모델 학습용 데이터 준비 완료.")
print(all_ratios_df.head())

✅ 모델 학습용 데이터 준비 완료.
  Ticker       Date  CurrentRatio  QuickRatio  GrossProfitMargin  \
0   AAPL 2024-09-30      0.867313    0.826007           0.462063   
1   AAPL 2023-09-30      0.988012    0.944442           0.441311   
2   AAPL 2022-09-30      0.879356    0.847235           0.433096   
3   AAPL 2021-09-30      1.074553    1.022115           0.417794   
5   MSFT 2025-06-30      1.353446    1.346804           0.688237   

   OperatingProfitMargin  DebtRatio  FinancialRisk  
0               0.315102   1.872327              0  
1               0.298214   1.787533              0  
2               0.302887   2.614462              0  
3               0.297824   2.163925              0  
5               0.456220   0.176395              0  


### **2단계: 데이터 분리 및 모델 훈련**

이제 준비된 데이터를 훈련용과 테스트용으로 나눈 뒤, 모델을 학습시킵니다.

In [11]:
# 독립 변수(X)와 종속 변수(y) 정의
# 독립 변수(X): 모델에 입력할 재무 비율들
# 종속 변수(y): 모델이 예측할 재무 위험 여부
X = all_ratios_df[['CurrentRatio', 'QuickRatio', 'GrossProfitMargin', 'OperatingProfitMargin', 'DebtRatio']]
y = all_ratios_df['FinancialRisk']

# 훈련 데이터와 테스트 데이터로 분리
# test_size=0.2는 전체 데이터의 20%를 테스트용으로 사용하겠다는 의미
# random_state는 결과를 재현 가능하게 함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 선택 및 훈련 (Random Forest 모델 사용)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("✅ 머신러닝 모델 훈련 완료.")

✅ 머신러닝 모델 훈련 완료.


### **3단계: 모델 성능 평가**

훈련된 모델이 얼마나 정확한지 평가 지표들을 통해 확인합니다.

In [12]:
# 테스트 데이터로 예측 수행
y_pred = model.predict(X_test)

# 성능 지표 계산
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

# 결과 출력
print("\n✅ 모델 성능 평가 결과")
print(f"정확도(Accuracy): {accuracy:.4f}")
print(f"정밀도(Precision): {precision:.4f}")
print(f"재현율(Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


✅ 모델 성능 평가 결과
정확도(Accuracy): 1.0000
정밀도(Precision): 0.0000
재현율(Recall): 0.0000
F1-Score: 0.0000


## **추가 학습: 프로젝트 진행 과정에서 마주친 핵심 머신러닝 이론 총정리**

### **1. 지도학습(Supervised Learning)의 이해**

이번 프로젝트에서 구축한 모델은 **지도학습**의 한 유형인 **분류(Classification)** 모델입니다.

* **지도학습이란?**: 정답(목표 변수, $y$)이 있는 데이터를 사용하여 모델을 학습시키는 방식입니다. 예를 들어, 재무 비율($X$)을 보고 '재무 위험 여부(0 또는 1)'라는 정답을 예측하도록 모델을 지도한 것입니다.

* **왜 `FinancialRisk`를 목표 변수로 정의했나?**

    머신러닝 모델은 추상적인 개념을 직접 예측할 수 없습니다. '재무 위험'은 복합적인 개념이므로, 이를 모델이 이해할 수 있는 구체적인 형태, 즉 **수치화된 정답($y$)**으로 변환해야 합니다. `FinancialRisk`를 **다음 해에 순손실이 발생했는가**라는 질문의 답(예: 1, 아니오: 0)으로 정의한 것은 바로 이 과정을 수행한 것입니다.

    이처럼 실무에서 정의하는 추상적인 개념을 머신러닝 모델이 예측할 수 있는 **구체적인 대리 변수**(Proxy Variable)로 변환하는 것은 **머신러닝 문제 설계를 위한 핵심적인 역량**입니다. 이 과정을 통해 여러분은 '데이터를 어떻게 활용하여 현실의 문제를 해결할 것인가'에 대한 명확한 방향을 설정할 수 있습니다.

* **지도학습의 2가지 유형**:
    1.  **분류(Classification)**: 결과가 이산적(Discrete)인 경우. '재무 위험 여부'처럼 예/아니오, A/B/C 등 범주형 결과를 예측합니다.
    2.  **회귀(Regression)**: 결과가 연속적(Continuous)인 경우. 주택 가격이나 기온처럼 숫자를 예측합니다.

### **2. 머신러닝 모델의 학습 과정**

모델은 마치 학생이 시험공부를 하는 것과 같습니다.

1.  **데이터 분리**: 전체 데이터를 **훈련 세트(Training Set)**와 **테스트 세트(Test Set)**로 나눕니다. 모델은 훈련 세트로 '공부'하고, 테스트 세트로 '시험'을 봅니다. 이렇게 하는 이유는 모델이 공부한 내용만 잘 맞추는 '과적합(Overfitting)' 현상을 방지하기 위해서입니다.

2.  **모델 훈련**: `model.fit(X_train, y_train)` 코드는 모델에게 훈련 데이터를 제공하여 패턴을 학습시키라는 명령입니다. 모델은 이 과정을 통해 재무 비율과 재무 위험 간의 관계를 스스로 찾아냅니다.

3.  **모델 평가**: `model.predict(X_test)`는 모델이 시험을 보는 과정입니다. 모델이 예측한 값($y_{pred}$)을 실제 정답($y_{test}$)과 비교하여 성능을 측정합니다.

### **3. 모델 평가 지표의 의미**

머신러닝 모델의 성능을 평가할 때는 단순히 '정확도(Accuracy)'만 봐서는 안 됩니다. 특히 재무 위험 예측과 같이 중요한 문제에서는 더 신중해야 합니다.

* **정확도(Accuracy)**: 전체 예측 중 맞은 예측의 비율. 가장 직관적이지만, 데이터가 한쪽으로 치우쳐 있을 때(예: 대부분의 기업이 정상)는 오해의 소지가 있습니다.

* **정밀도(Precision)**: 모델이 **'재무 위험'이라고 예측한 것들 중** 실제로 재무 위험이었던 비율입니다. **오류가 적은 예측**이 중요할 때 사용합니다. 감사 관점에서 정밀도가 높다는 것은 불필요한 감사 대상(정상 기업)을 '재무 위험'으로 잘못 분류하는 경우가 적다는 의미입니다.

* **재현율(Recall)**: **실제로 재무 위험이었던 기업들 중** 모델이 올바르게 예측한 비율입니다. **놓치지 않고 찾아내는 것**이 중요할 때 사용합니다. 재현율이 높다는 것은 실제로 위험한 기업을 '정상'으로 잘못 분류하는 위험(감사 누락)을 줄일 수 있다는 의미입니다.

* **F1-Score**: 정밀도와 재현율의 조화 평균으로, 두 지표를 모두 고려한 균형 잡힌 평가 지표입니다.

이러한 평가 지표들은 감사인이 모델의 예측 결과를 어떻게 신뢰하고 활용할지 결정하는 데 중요한 근거를 제공합니다.
